In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [ ]:
keras = tf.keras
Input = keras.layers.Input
Conv2D = keras.layers.Conv2D
Flatten = keras.layers.Flatten
Dense = keras.layers.Dense
GlobalAveragePooling2D = keras.layers.GlobalAveragePooling2D
Model = keras.Model

In [ ]:
train_data = np.load('./data/train_data.npy')
train_labels = np.load('./data/train_labels.npy')

validation_data = np.load('./data/validation_data.npy')
validation_labels = np.load('./data/validation_labels.npy')

test_data = np.load('./data/test_data.npy')
test_labels = np.load('./data/test_labels.npy')

In [ ]:
# One-hot encode the labels
PATH_TO_DATA = os.path.join(os.getcwd(), 'data')
PATH_TO_AUDIO = os.path.join(PATH_TO_DATA, 'audio')
LABELS = os.listdir(PATH_TO_AUDIO)
NUM_CLASSES = len(LABELS)
label_encoder = LabelEncoder().fit(LABELS)

train_labels = keras.utils.to_categorical(label_encoder.transform(train_labels), num_classes=NUM_CLASSES)
validation_labels = keras.utils.to_categorical(label_encoder.transform(validation_labels), num_classes=NUM_CLASSES)
test_labels = keras.utils.to_categorical(label_encoder.transform(test_labels), num_classes=NUM_CLASSES)

In [ ]:
# Reshape data for compatibility with Keras
train_data = train_data.reshape((*train_data.shape, 1))
validation_data = validation_data.reshape((*validation_data.shape, 1))
test_data = test_data.reshape((*test_data.shape, 1))

#### Basic CNN Model

In [ ]:
# tf.reset_default_graph()

In [ ]:
input_layer = Input(shape=train_data.shape[1:])
conv_1 = Conv2D(filters=48, kernel_size=(8, 3), activation='relu')(input_layer)
conv_2 = Conv2D(filters=48, kernel_size=(8, 3), activation='relu')(conv_1)
conv_3 = Conv2D(filters=36, kernel_size=(8, 3), activation='relu')(conv_2)
conv_4 = Conv2D(filters=36, kernel_size=(8, 3), activation='relu')(conv_3)
# gap_1 = GlobalAveragePooling2D()(conv_4)
flatten = Flatten()(conv_4)
dense = Dense(30, activation='softmax')(flatten)
model = Model(inputs=input_layer, outputs=dense)

# Display model summary for reference
model.summary()

Define callbacks

In [ ]:
early_stop_callback = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                    min_delta=0, 
                                                    patience=10, 
                                                    verbose=0, 
                                                    mode='auto')
reduce_lr_plateau_callback = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                               factor=0.1, 
                                                               patience=5, 
                                                               verbose=0, 
                                                               mode='auto', 
                                                               cooldown=0, min_lr=0)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, 
          train_labels, 
          epochs=100, 
          batch_size=1024, 
          validation_data=(validation_data, validation_labels), 
          shuffle=True,
          callbacks=[early_stop_callback, reduce_lr_plateau_callback],
          verbose=1)

In [ ]:
model.save('arch2_100e_1024b')